In [1]:
#!/bin/python
"""
Tutorial to demonstrate a new interferometer

We place a new instrument in Gingin, with an A+ sensitivity in a network of A+
interferometers at Hanford and Livingston
"""

'\nTutorial to demonstrate a new interferometer\n\nWe place a new instrument in Gingin, with an A+ sensitivity in a network of A+\ninterferometers at Hanford and Livingston\n'

In [2]:
import numpy as np

import bilby
#import gwinc

# Set the duration and sampling frequency of the data segment that we're going
# to inject the signal into
duration = 4.
sampling_frequency = 2048.

In [3]:
# Specify the output directory and the name of the simulation.
outdir = 'outdir'
label = 'next_genBBH'
bilby.core.utils.setup_logger(outdir=outdir, label=label)

# Set up a random seed for result reproducibility.  This is optional!
np.random.seed(88170232)

AdV_psd=np.loadtxt("AdV_psd.txt")
AdV_psd=AdV_psd[AdV_psd[:,0]>=20]

hl_psd=np.loadtxt("aLIGO_ZERO_DET_high_P_psd.txt")
hl_psd=hl_psd[hl_psd[:,0]>=20]


frequencies = np.logspace(0, 3, 1000)

In [4]:
# Set up three other detectors at Hanford, Livingston and Virgo
interferometers = bilby.gw.detector.InterferometerList(['H1', 'L1','V1'])

In [9]:
interferometers[0].power_spectral_density =\
        bilby.gw.detector.PowerSpectralDensity(
            frequency_array=hl_psd[:,0], psd_array=hl_psd[:,1])

interferometers[1].power_spectral_density =\
        bilby.gw.detector.PowerSpectralDensity(
            frequency_array=hl_psd[:,0], psd_array=hl_psd[:,1])
    
interferometers[2].power_spectral_density =\
        bilby.gw.detector.PowerSpectralDensity(
            frequency_array=AdV_psd[:,0], psd_array=AdV_psd[:,1])

In [10]:
 # Inject a gravitational-wave signal into the network
# as we're using a three-detector network of A+, we inject a GW150914-like
# signal at 4 Gpc
injection_parameters = dict(
    mass_1=36., mass_2=29., a_1=0.4, a_2=0.3, tilt_1=0.5, tilt_2=1.0,
    phi_12=1.7, phi_jl=0.3, luminosity_distance=2000., theta_jn=0.4, psi=2.659,
    phase=1.3, geocent_time=1502019943, ra=1.375, dec=-1.2108)
# date and time 2027-08-11T11:45:25

In [11]:
# Fixed arguments passed into the source model
waveform_arguments = dict(waveform_approximant='IMRPhenomPv2',
                          reference_frequency=50.)

# Create the waveform_generator using a LAL BinaryBlackHole source function
waveform_generator = bilby.gw.WaveformGenerator(
    duration=duration, sampling_frequency=sampling_frequency,
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    waveform_arguments=waveform_arguments)

start_time = injection_parameters['geocent_time'] + 2 - duration

09:51 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


In [12]:
# inject the signal into the interferometers

for interferometer in interferometers:
    interferometer.set_strain_data_from_power_spectral_density(
        sampling_frequency=sampling_frequency, duration=duration, start_time=start_time)
    interferometer.inject_signal(
        parameters=injection_parameters, waveform_generator=waveform_generator)

    # plot the data for sanity
    signal = interferometer.get_detector_response(
        waveform_generator.frequency_domain_strain(), injection_parameters)
    interferometer.plot_data(signal=signal, outdir=outdir, label=label)

09:51 bilby INFO    : Injected signal in H1:
09:51 bilby INFO    :   optimal SNR = 12.35
09:51 bilby INFO    :   matched filter SNR = 11.96+0.24j
09:51 bilby INFO    :   mass_1 = 36.0
09:51 bilby INFO    :   mass_2 = 29.0
09:51 bilby INFO    :   a_1 = 0.4
09:51 bilby INFO    :   a_2 = 0.3
09:51 bilby INFO    :   tilt_1 = 0.5
09:51 bilby INFO    :   tilt_2 = 1.0
09:51 bilby INFO    :   phi_12 = 1.7
09:51 bilby INFO    :   phi_jl = 0.3
09:51 bilby INFO    :   luminosity_distance = 2000.0
09:51 bilby INFO    :   theta_jn = 0.4
09:51 bilby INFO    :   psi = 2.659
09:51 bilby INFO    :   phase = 1.3
09:51 bilby INFO    :   geocent_time = 1502019943
09:51 bilby INFO    :   ra = 1.375
09:51 bilby INFO    :   dec = -1.2108
09:51 bilby INFO    : Injected signal in L1:
09:51 bilby INFO    :   optimal SNR = 9.92
09:51 bilby INFO    :   matched filter SNR = 11.67+0.52j
09:51 bilby INFO    :   mass_1 = 36.0
09:51 bilby INFO    :   mass_2 = 29.0
09:51 bilby INFO    :   a_1 = 0.4
09:51 bilby INFO    

In [13]:
priors = bilby.gw.prior.BBHPriorDict()
for key in ['a_1', 'a_2', 'tilt_1', 'tilt_2', 'phi_12', 'phi_jl', 'psi',
            'geocent_time']:
    priors[key] = injection_parameters[key]
   
priors['geocent_time'] = bilby.core.prior.Uniform(
    minimum=injection_parameters['geocent_time'] - 0.1,
    maximum=injection_parameters['geocent_time'] + 0.1,
    name='geocent_time', latex_label='$t_c$', unit='$s$')

09:51 bilby INFO    : No prior given, using default BBH priors in /Users/dhruv/miniconda3/envs/igwn-py39/lib/python3.9/site-packages/bilby/gw/prior_files/precessing_spins_bbh.prior.


In [14]:
# Initialise the likelihood by passing in the interferometer data (IFOs)
# and the waveoform generator
likelihood = bilby.gw.GravitationalWaveTransient(
    interferometers=interferometers, waveform_generator=waveform_generator,
    time_marginalization=True, phase_marginalization=True,
    distance_marginalization=True, priors=priors)

09:51 bilby INFO    : Distance marginalisation file .distance_marginalization_lookup.npz does not exist
09:51 bilby INFO    : Building lookup table for distance marginalisation.


  0%|          | 0/400 [00:00<?, ?it/s]

In [15]:
result = bilby.run_sampler(
    likelihood=likelihood, priors=priors, sampler='dynesty', npoints=2000,
    injection_parameters=injection_parameters, outdir=outdir,
    label=label, maxmcmc=1000, nlive=512, walks=100, n_checkpoint=5000, dlogz=0.2,
    conversion_function=bilby.gw.conversion.generate_all_bbh_parameters)

# make some plots of the outputs
result.plot_corner()

09:52 bilby INFO    : Running for label 'next_genBBH', output will be saved to 'outdir'
09:52 bilby INFO    : Using lal version 7.2.4
09:52 bilby INFO    : Using lal git version Branch: None;Tag: lalsuite-v7.11;Id: 7a2f2aa176ad39eeaede38f6df4a41d6bf226e8f;;Builder: Unknown User <>;Repository status: CLEAN: All modifications committed
09:52 bilby INFO    : Using lalsimulation version 4.0.2
09:52 bilby INFO    : Using lalsimulation git version Branch: None;Tag: lalsuite-v7.11;Id: 7a2f2aa176ad39eeaede38f6df4a41d6bf226e8f;;Builder: Unknown User <>;Repository status: CLEAN: All modifications committed
09:52 bilby INFO    : Analysis priors:
09:52 bilby INFO    : mass_ratio=bilby.gw.prior.UniformInComponentsMassRatio(minimum=0.125, maximum=1, name='mass_ratio', latex_label='$q$', unit=None, boundary=None, equal_mass=False)
09:52 bilby INFO    : chirp_mass=bilby.gw.prior.UniformInComponentsChirpMass(minimum=25, maximum=100, name='chirp_mass', latex_label='$\\mathcal{M}$', unit=None, boundary=N

1it [00:00, ?it/s]

09:52 bilby INFO    : Run interrupted by signal 2: checkpoint and exit on 130
09:52 bilby INFO    : Written checkpoint file outdir/next_genBBH_resume.pickle


Exception while calling loglikelihood function:
  params: [6.84258680e-01 6.28348272e+01 1.53950337e+00 2.69868197e+00
 1.35318173e+00 2.48175140e-04]
  args: []
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "/Users/dhruv/miniconda3/envs/igwn-py39/lib/python3.9/site-packages/dynesty/dynesty.py", line 910, in __call__
    return self.func(np.asarray(x).copy(), *self.args, **self.kwargs)
  File "/Users/dhruv/miniconda3/envs/igwn-py39/lib/python3.9/site-packages/bilby/core/sampler/dynesty.py", line 53, in _log_likelihood_wrapper
    return _sampling_convenience_dump.likelihood.log_likelihood_ratio()
  File "/Users/dhruv/miniconda3/envs/igwn-py39/lib/python3.9/site-packages/bilby/gw/likelihood/base.py", line 394, in log_likelihood_ratio
    self.waveform_generator.frequency_domain_strain(self.parameters)
  File "/Users/dhruv/miniconda3/envs/igwn-py39/lib/python3.9/site-packages/bilby/gw/waveform_generator.py", line 131, in frequency_domain_strain
    return self._calculate_strain(model=self.frequency_domain_source_model,
  File "/Users/dhruv/miniconda3/envs/igwn-py39/lib/python3.9/site-packages/bilby/gw/waveform_generator.py", line 176, in _calculate_s

SystemExit: 130

/Users/dhruv/miniconda3/envs/igwn-py39/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
